In [176]:
import numpy as np
from scipy.sparse.linalg import eigsh
import scipy.sparse as sparse
from scipy.linalg import eigh
import scipy.linalg
import matplotlib.pyplot as plt
from qutip import *
import math

In [177]:
#plt.rcParams['figure.figsize']=8.5,5.1
plt.rcParams['font.size']=20
plt.rcParams['lines.linewidth']=1.5
plt.rcParams['text.usetex']=True

In [178]:
#Construction of necessary elements of theta and phi Hilbert spaces

# For phi

N=50
a_phi=np.diag(np.sqrt(np.arange(1, N+1)), 1)
ad_phi=np.diag(np.sqrt(np.arange(1, N+1)), -1)
N_phi=ad_phi@a_phi
Id_N=np.eye(N+1)

# For theta

M=20
Id_M=np.eye(2*M+1)
n_theta=np.flipud(np.fliplr(np.diag(np.arange(-M,M+1), 0)))

#Values of equilibrium

phi_ext_val=np.pi
ng_val=0.5

#Qubit parameters 1 

E_Cs=1.0
E_Cj=1.0
E_L=1.0
E_J=1.0
alpha=(1j*(E_Cj/E_L)**(1.0/4.0))

# Qubit parameters 2

E_Cs_2=(1.0/20)
E_Cj_2=1.0
E_L_2=(1.0/16)
E_J_2=(1.0/5.0)
alpha2=(1j*(E_Cj_2/E_L_2)**(1.0/4.0))



In [216]:
#1ST VALUES

#ng1=np.arange(0.495,0.506,0.01)
epsilon_n=0.005
#print(ng1)
#a=np.pi-(np.pi*0.01)
#b=np.pi+(np.pi*0.01 + 0.001)
#phi_ext1=np.arange(a,b,(0.01*np.pi)*2)
epsilon_phi=(0.01*np.pi)
#print(phi_ext1)

#2ND VALUES

#ng2=np.arange(0.495,0.506,0.01)
#phi_ext2=np.arange(a,b,(0.01*np.pi)*2)

T=50*1/0.083
N_time=100
Delta_t=T/N_time
time_array=np.linspace(Delta_t,T,N_time)
print(time_array)

[  6.02409639  12.04819277  18.07228916  24.09638554  30.12048193
  36.14457831  42.1686747   48.19277108  54.21686747  60.24096386
  66.26506024  72.28915663  78.31325301  84.3373494   90.36144578
  96.38554217 102.40963855 108.43373494 114.45783133 120.48192771
 126.5060241  132.53012048 138.55421687 144.57831325 150.60240964
 156.62650602 162.65060241 168.6746988  174.69879518 180.72289157
 186.74698795 192.77108434 198.79518072 204.81927711 210.84337349
 216.86746988 222.89156627 228.91566265 234.93975904 240.96385542
 246.98795181 253.01204819 259.03614458 265.06024096 271.08433735
 277.10843373 283.13253012 289.15662651 295.18072289 301.20481928
 307.22891566 313.25301205 319.27710843 325.30120482 331.3253012
 337.34939759 343.37349398 349.39759036 355.42168675 361.44578313
 367.46987952 373.4939759  379.51807229 385.54216867 391.56626506
 397.59036145 403.61445783 409.63855422 415.6626506  421.68674699
 427.71084337 433.73493976 439.75903614 445.78313253 451.80722892
 457.831325

In [180]:
# Construction of the 0-pi Hamiltonian

def timedep_Hamiltonian_0pi_for_ng(EJ,ECS,ECJ,EL,n,phi,alph,time):
    #Initial
    H=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #print("H =>",len(H))
    #Free Hamiltonian construction
    omega=np.pi/T
    n_t=n-epsilon_n*np.cos(omega*time)
    H1=4*ECS*((n_theta-n_t*Id_M)@(n_theta-n_t*Id_M))
    H_theta=np.kron(H1,Id_N)
    #print("H_theta =>",len(H_theta))
    H2=2*np.sqrt(ECJ*EL)*(N_phi+0.5*Id_N)
    H_phi=np.kron(Id_M,H2)
    #print("H_phi =>",len(H_phi))
    H_0=H_theta+H_phi
    #print("H_0 =>",len(H_0))
    #H_0=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Interaction Hamiltonian construction
    cos_theta_ext=np.diag(np.ones(2*M),1)+np.diag(np.ones(2*M),-1) 
    #cos_theta=np.diag(np.ones(2*M-1),2)+np.diag(np.ones(2*M-1),-2)
    cos_theta_ext[0][2*M]+=1
    cos_theta_ext[2*M][0]+=1
    phi_ext_t=phi-epsilon_phi*np.sin(omega*time)
    #cos_phi=(0.5*((displace(N+1,alph)*(math.e**(-1j*phi/2)))+(displace(N+1,-alph)*(math.e**(1j*phi/2)))))*np.diag(np.ones(N+1),0)
    cos_phi=0.5*(scipy.linalg.expm(alph*(a_phi+ad_phi)-(Id_N*1j*phi_ext_t/2))+scipy.linalg.expm(-alph*(a_phi+ad_phi)+(Id_N*1j*phi_ext_t/2)))
    H_I=EJ*np.kron(cos_theta_ext,cos_phi.real)#SIN EL2 
    #print("H_I =>",len(H_I))
    #H_I=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Total Hamiltonian
    #print("H =>",len(H))
    H+=H_0-H_I
    #print("H =>",len(H))
    return H

In [181]:
# Construction of the 0-pi Hamiltonian

def timedep_Hamiltonian_0pi_for_phiext(EJ,ECS,ECJ,EL,n,phi,alph,time):
    #Initial
    H=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Free Hamiltonian construction
    omega=np.pi/T
    n_t=n-epsilon_n*np.sin(omega*time)
    H1=4*ECS*((n_theta-n_t*Id_M)@(n_theta-n_t*Id_M))
    H_theta=np.kron(H1,Id_N)
    H2=2*np.sqrt(ECJ*EL)*(N_phi+0.5*Id_N)
    H_phi=np.kron(Id_M,H2)
    H_0=H_theta+H_phi
    #H_0=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Interaction Hamiltonian construction
    cos_theta_ext=np.diag(np.ones(2*M),1)+np.diag(np.ones(2*M),-1) 
    #cos_theta=np.diag(np.ones(2*M-1),2)+np.diag(np.ones(2*M-1),-2)
    cos_theta_ext[0][2*M]+=1
    cos_theta_ext[2*M][0]+=1
    phi_ext_t=phi-epsilon_phi*np.cos(omega*time)
    #cos_phi=(0.5*((displace(N+1,alph)*(math.e**(-1j*phi/2)))+(displace(N+1,-alph)*(math.e**(1j*phi/2)))))*np.diag(np.ones(N+1),0)
    cos_phi=0.5*(scipy.linalg.expm(alph*(a_phi+ad_phi)-(Id_N*1j*phi_ext_t/2))+scipy.linalg.expm(-alph*(a_phi+ad_phi)+(Id_N*1j*phi_ext_t/2)))
    H_I=EJ*np.kron(cos_theta_ext,cos_phi.real)#SIN EL2 
    #H_I=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Total Hamiltonian
    H+=H_0-H_I
    return H

In [182]:
def U_t(Hamiltonian,time):
    return scipy.linalg.expm(-1j*Hamiltonian*time)

In [183]:
def calculate_overlap(vect1,vect2):
    return np.abs(np.conj(vect1)@vect2)**2

In [217]:
#THESE ARE LOAD FOR M=20,N=50
EIGENVECTS_NG=np.load('EIGVECTS_NG.npy')
EIGENVECTS_PHIEXT=np.load('EIGVECTS_PHIEXT.npy')
EIGENVECTS2_NG=np.load('EIGVECTS2_NG.npy')
EIGENVECTS2_PHIEXT=np.load('EIGVECTS2_PHIEXT.npy')
#print("SET 1 FOR NG =>",EIGENVECTS_NG)
#print("SET 1 FOR PHI_EXT =>",EIGENVECTS_PHIEXT)
#print("SET 2 FOR NG =>",EIGENVECTS2_NG)
#print("SET 2 FOR PHI_EXT =>",EIGENVECTS2_PHIEXT)

In [218]:
Psi2_GS_NG=EIGENVECTS2_NG[0]
#print(len(Psi_GS_NG))

for t in time_array:
    #DEFINITION OF THE AUXILIARY HAMILTONIAN
    #ng
    Haux2_NG=timedep_Hamiltonian_0pi_for_ng(E_J_2,E_Cs_2,E_Cj_2,E_L_2,ng_val,phi_ext_val,alpha2,t)
    #print(len(Haux_NG))
    print("Hamiltonian calculated")
    #CONSTRUCTION OF THE PROPAGATORS
    #ng
    propagator2_NG=U_t(Haux2_NG,Delta_t)
    #print(len(Haux_NG))
    print("Propagator calculated")
    #EVOLUTION OF THE STATE FOR EACH TIME STEP
    #ng
    Psi2_GS_NG=propagator2_NG@Psi2_GS_NG
    print("Evolution step = ",t," done")
    #OVERLAP FOR CHECKING THE "FIDELITY" OF THE NEW STATES WITH RESPECT TO THE GOAL ONES
    #ng
    Overlap2_Psi_GS_NG=calculate_overlap(Psi2_GS_NG,EIGENVECTS2_NG[2])
    print("Overlap calculated")
    print("Overlap_Psi_GS_NG =",Overlap2_Psi_GS_NG)

Hamiltonian calculated
Propagator calculated
Evolution step =  6.024096385542168  done
Overlap calculated
Overlap_Psi_GS_NG = 6.565967757052287e-09
Hamiltonian calculated
Propagator calculated
Evolution step =  12.048192771084336  done
Overlap calculated
Overlap_Psi_GS_NG = 5.909114417925008e-08
Hamiltonian calculated
Propagator calculated
Evolution step =  18.072289156626503  done
Overlap calculated
Overlap_Psi_GS_NG = 2.3609986658862267e-07
Hamiltonian calculated
Propagator calculated
Evolution step =  24.096385542168672  done
Overlap calculated
Overlap_Psi_GS_NG = 6.549430721820141e-07
Hamiltonian calculated
Propagator calculated
Evolution step =  30.12048192771084  done
Overlap calculated
Overlap_Psi_GS_NG = 1.4711796614229011e-06
Hamiltonian calculated
Propagator calculated
Evolution step =  36.14457831325301  done
Overlap calculated
Overlap_Psi_GS_NG = 2.8774578210326584e-06
Hamiltonian calculated
Propagator calculated
Evolution step =  42.16867469879517  done
Overlap calculated


In [214]:
Psi_GS_NG=EIGENVECTS_NG[0]
Psi_1st_NG=EIGENVECTS_NG[1]
Psi2_GS_NG=EIGENVECTS2_NG[0]
Psi2_1st_NG=EIGENVECTS2_NG[1]
Psi_GS_PHIEXT=EIGENVECTS_NG[0]
Psi_1st_PHIEXT=EIGENVECTS_NG[1]
Psi2_GS_PHIEXT=EIGENVECTS2_NG[0]
Psi2_1st_PHIEXT=EIGENVECTS2_NG[1]

for t in time_array:
    #DEFINITION OF THE AUXILIARY HAMILTONIAN
    #ng
    Haux_NG=timedep_Hamiltonian_0pi_for_ng(E_J,E_Cs,E_Cj,E_L,ng_val,phi_ext_val,alpha,t)
    Haux2_NG=timedep_Hamiltonian_0pi_for_ng(E_J_2,E_Cs_2,E_Cj_2,E_L_2,ng_val,phi_ext_val,alpha2,t)
    #phi_ext
    Haux_PHIEXT=timedep_Hamiltonian_0pi_for_phiext(E_J,E_Cs,E_Cj,E_L,ng_val,phi_ext_val,alpha,t)
    Haux2_PHIEXT=timedep_Hamiltonian_0pi_for_phiext(E_J_2,E_Cs_2,E_Cj_2,E_L_2,ng_val,phi_ext_val,alpha2,t)
    print("Hamiltonians calculated")
    #CONSTRUCTION OF THE PROPAGATORS
    #ng
    propagator_NG=U_t(Haux_NG,Delta_t)
    propagator2_NG=U_t(Haux2_NG,Delta_t)
    #phi_ext
    propagator_PHIEXT=U_t(Haux_PHIEXT,Delta_t)
    propagator2_PHIEXT=U_t(Haux2_PHIEXT,Delta_t)
    print("Propagators calculated")
    #EVOLUTION OF THE STATE FOR EACH TIME STEP
    #ng
    Psi_GS_NG=propagator_NG@Psi_GS_NG
    Psi_1st_NG=propagator_NG@Psi_1st_NG
    Psi2_GS_NG=propagator2_NG@Psi2_GS_NG
    Psi2_1st_NG=propagator2_NG@Psi2_1st_NG
    #phi_ext
    Psi_GS_PHIEXT=propagator_PHIEXT@Psi_GS_PHIEXT
    Psi_1st_PHIEXT=propagator_PHIEXT@Psi_1st_PHIEXT
    Psi2_GS_PHIEXT=propagator2_PHIEXT@Psi2_GS_PHIEXT
    Psi2_1st_PHIEXT=propagator2_PHIEXT@Psi2_1st_PHIEXT
    print("Evolution step = ",t," done")
    #OVERLAP FOR CHECKING THE "FIDELITY" OF THE NEW STATES WITH RESPECT TO THE GOAL ONES
    #ng
    Overlap_Psi_GS_NG=calculate_overlap(Psi_GS_NG,EIGENVECTS_NG[2])
    Overlap_Psi_1st_NG=calculate_overlap(Psi_1st_NG,EIGENVECTS_NG[3])
    Overlap_Psi2_GS_NG=calculate_overlap(Psi2_GS_NG,EIGENVECTS2_NG[2])
    Overlap_Psi2_1st_NG=calculate_overlap(Psi2_1st_NG,EIGENVECTS2_NG[3])
    #phi_ext
    Overlap_Psi_GS_PHIEXT=calculate_overlap(Psi_GS_PHIEXT,EIGENVECTS_PHIEXT[2])
    Overlap_Psi_1st_PHIEXT=calculate_overlap(Psi_1st_PHIEXT,EIGENVECTS_PHIEXT[3])
    Overlap_Psi2_GS_PHIEXT=calculate_overlap(Psi2_GS_PHIEXT,EIGENVECTS2_PHIEXT[2])
    Overlap_Psi2_1st_PHIEXT=calculate_overlap(Psi2_1st_PHIEXT,EIGENVECTS2_PHIEXT[3])
    print("Overlaps calculated")
    print("Overlap_Psi_GS_NG =",Overlap_Psi_GS_NG)
    print("Overlap_Psi_GS_NG =",Overlap_Psi_1st_NG)
    print("Overlap_Psi_GS_NG =",Overlap_Psi2_GS_NG)
    print("Overlap_Psi_GS_NG =",Overlap_Psi2_1st_NG)
    print("Overlap_Psi_GS_NG =",Overlap_Psi_GS_PHIEXT)
    print("Overlap_Psi_GS_NG =",Overlap_Psi_1st_PHIEXT)
    print("Overlap_Psi_GS_NG =",Overlap_Psi2_GS_PHIEXT)
    print("Overlap_Psi_GS_NG =",Overlap_Psi2_1st_PHIEXT)
    

Hamiltonians calculated
Propagators calculated
Evolution step =  3.542168674698795  done
Overlaps calculated
Overlap_Psi_GS_NG = 9.534068806042656e-07
Overlap_Psi_GS_NG = 9.519126298410005e-07
Overlap_Psi_GS_NG = 2.269217246822147e-09
Overlap_Psi_GS_NG = 2.2739800401051187e-09
Overlap_Psi_GS_NG = 0.4999082299938814
Overlap_Psi_GS_NG = 0.49990620673554903
Overlap_Psi_GS_NG = 0.4999877203554254
Overlap_Psi_GS_NG = 0.49998770746985594
Hamiltonians calculated
Propagators calculated
Evolution step =  7.08433734939759  done
Overlaps calculated
Overlap_Psi_GS_NG = 8.543967620411259e-06
Overlap_Psi_GS_NG = 8.538286852962809e-06
Overlap_Psi_GS_NG = 2.0418313167575614e-08
Overlap_Psi_GS_NG = 2.0442884426370272e-08
Overlap_Psi_GS_NG = 0.49954293521755555
Overlap_Psi_GS_NG = 0.4995414666772778
Overlap_Psi_GS_NG = 0.499987699248588
Overlap_Psi_GS_NG = 0.4999877221197249
Hamiltonians calculated
Propagators calculated
Evolution step =  10.626506024096384  done
Overlaps calculated
Overlap_Psi_GS_NG = 

In [ ]:
#CONSTRUCTION OF THE PROPAGATORS
    #ng
    propagator_NG=scipy.linlg.expm(1j*Delta_t*Haux_NG)
    propagator2_NG=scipy.linlg.expm(1j*Delta_t*Haux2_NG)
    #phi_ext
    propagator_PHIEXT=scipy.linlg.expm(1j*Delta_t*Haux_PHIEXT)
    propagator2_PHIEXT=scipy.linlg.expm(1j*Delta_t*Haux2_PHIEXT)
    print("Propagators calculated")
    #EVOLUTION OF THE STAATE FOR EACH TIME STEP
    #ng
    Psi_GS_NG=propagator_NG@Psi_GS_NG
    Psi_1st_NG=propagator_NG@Psi_1st_NG
    Psi2_GS_NG=propagator2_NG@Psi2_GS_NG
    Psi2_1st_NG=propagator2_NG@Psi2_1st_NG
    #phi_ext
    Psi_GS_PHIEXT=propagator_PHIEXT@Psi_GS_PHIEXT
    Psi_1st_PHIEXT=propagator_PHIEXT@Psi_1st_PHIEXT
    Psi2_GS_PHIEXT=propagator2_PHIEXT@Psi2_GS_PHIEXT
    Psi2_1st_PHIEXT=propagator2_PHIEXT@Psi2_1st_PHIEXT
    print("Evolution step = ",t," done")
    #OVERLAP FOR CHECKING THE "FIDELITY" OF THE NEW STATES WITH RESPECT TO THE GOAL ONES
    #ng
    Overlap_Psi_GS_NG=calculate_overlap(Psi_GS_NG,EIGENVECTS_NG[2])
    Overlap_Psi_1st_NG=calculate_overlap(Psi_1st_NG,EIGENVECTS_NG[3])
    Overlap_Psi2_GS_NG=calculate_overlap(Psi2_GS_NG,EIGENVECTS2_NG[2])
    Overlap_Psi2_1st_NG=calculate_overlap(Psi2_1st_NG,EIGENVECTS2_NG[3])
    #phi_ext
    Overlap_Psi_GS_PHIEXT=calculate_overlap(Psi_GS_PHIEXT,EIGENVECTS_PHIEXT[2])
    Overlap_Psi_1st_PHIEXT=calculate_overlap(Psi_1st_PHIEXT,EIGENVECTS_PHIEXT[3])
    Overlap_Psi2_GS_PHIEXT=calculate_overlap(Psi2_GS_PHIEXT,EIGENVECTS2_PHIEXT[2])
    Overlap_Psi2_1st_PHIEXT=calculate_overlap(Psi2_1st_PHIEXT,EIGENVECTS2_PHIEXT[3])
    print("Overlaps calculated")
    print("Overlap_Psi_GS_NG =",Overlap_Psi_GS_NG)
    print("Overlap_Psi_GS_NG =",Overlap_Psi_1st_NG)
    print("Overlap_Psi_GS_NG =",Overlap_Psi2_GS_NG)
    print("Overlap_Psi_GS_NG =",Overlap_Psi2_1st_NG)
    print("Overlap_Psi_GS_NG =",Overlap_Psi_GS_PHIEXT)
    print("Overlap_Psi_GS_NG =",Overlap_Psi_1st_PHIEXT)
    print("Overlap_Psi_GS_NG =",Overlap_Psi2_GS_PHIEXT)
    print("Overlap_Psi_GS_NG =",Overlap_Psi2_1st_PHIEXT)

